In [41]:
import random
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from pricer.evaluator import evaluate
from pricer.items import Item

In [2]:
LITE_MODE=False

In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

README.md:   0%|          | 0.00/744 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/243M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


validation-00000-of-00001.parquet:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00001.parquet:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test)} test items")

Loaded 800,000 training items, 10,000 validation items, 10000 test items


In [5]:
def random_pricer(item):
    return random.randrange(1, 1000)

In [6]:
random.seed(42)
evaluate(random_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$436 $1 $29 $690 $252 $21 $85 $72 $719 $225 $20 $380 $894 $505 $11 $572 $354 $17 $179 $23 $90 $115 $433 $442 $304 $122 $291 $714 $567 $639 $539 $370 $66 $380 $489 $534 $769 $835 $207 $740 $626 $84 $680 $178 $129 $260 $142 $189 $836 $580 $310 $25 $380 $270 $47 $234 $861 $313 $417 $259 $591 $33 $657 $361 $79 $38 $757 $500 $263 $5 $534 $284 $570 $625 $584 $871 $759 $361 $575 $178 $602 $60 $17 $579 $207 $732 $115 $224 $756 $193 $866 $9 $370 $250 $456 $423 $821 $217 $103 $195 $264 $98 $650 $135 $470 $842 $675 $264 $43 $325 $591 $138 $516 $619 $56 $2 $449 $369 $221 $845 $640 $616 $501 $59 $502 $273 $844 $688 $616 $81 $164 $705 $52 $795 $259 $396 $70 $2 $89 $798 $902 $331 $818 $716 $129 $186 $627 $2 $141 $873 $918 $553 $423 $7 $253 $136 $204 $707 $255 $502 $19 $739 $557 $426 $80 $575 $39 $321 $185 $132 $497 $484 $326 $751 $53 $733 $85 $64 $549 $71 $158 $672 $133 $362 $16 $373 $258 $544 $420 $515 $223 $944 $532 $743 $866 $68 $527 $459 $67 $673 

In [8]:
# Average pricing accross training data
training_prices = [item.price for item in train]
training_average = sum(training_prices) / len(training_prices)
print(training_average)

def constant_pricer(item):
    return training_average

140.56967545


In [12]:
evaluate(constant_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$78 $25 $86 $71 $111 $89 $4 $75 $105 $189 $572 $238 $121 $86 $61 $108 $61 $91 $70 $22 $7 $17 $56 $34 $191 $312 $354 $121 $42 $61 $121 $81 $19 $60 $25 $678 $81 $85 $73 $103 $59 $61 $106 $114 $79 $116 $123 $109 $5 $61 $105 $11 $334 $21 $87 $6 $134 $101 $62 $129 $95 $63 $50 $31 $488 $51 $99 $304 $16 $65 $109 $124 $139 $122 $91 $105 $16 $131 $124 $122 $21 $129 $111 $42 $114 $81 $42 $165 $21 $95 $119 $46 $121 $106 $132 $88 $107 $17 $129 $434 $41 $24 $104 $2 $108 $23 $116 $259 $110 $158 $81 $174 $110 $12 $55 $29 $116 $121 $85 $38 $125 $52 $70 $25 $59 $81 $121 $42 $38 $2 $69 $3 $55 $111 $76 $126 $64 $71 $12 $2 $76 $109 $60 $121 $53 $109 $96 $369 $124 $108 $122 $35 $94 $1 $121 $138 $92 $85 $179 $91 $148 $115 $99 $128 $699 $118 $307 $91 $101 $131 $116 $119 $279 $118 $39 $8 $113 $48 $46 $48 $514 $116 $159 $108 $91 $119 $7 $74 $81 $114 $106 $90 $106 $2 $41 $61 $29 $140 $90 $115 

In [22]:
def get_features(item):
    return {
        # True item wieght in pounds
        "weight": item.weight,
        "weight_unknown": 1 if item.weight==0 else 0,
        "text_length": len(item.summary)
    }

In [23]:
def list_to_dataframe(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in items]
    return df

In [24]:
train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test)

In [25]:
np.random.seed(42)

# Separate features and target
feature_columns = ['weight', 'weight_unknown', 'text_length']

X_train = train_df[feature_columns]
y_train = train_df['price']
X_test = test_df[feature_columns]
y_test = test_df['price']

# Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(feature_columns, model.coef_):
    print(f"{feature}:{coef}")
print(f"Intercept: {model.intercept_}")

# Predict the test set and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

weight:0.4486789805756249
weight_unknown:-6.627998877825424
text_length:0.24694518955630543
Intercept: 51.110998225441534
Mean Squared Error: 25615.84348857242
R-squared Score: -0.05946814914318388


In [26]:
def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    return model.predict(features_df)[0]

In [27]:
evaluate(linear_regression_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$55 $29 $87 $90 $92 $60 $1 $78 $104 $192 $556 $228 $141 $81 $50 $124 $69 $83 $50 $8 $10 $9 $61 $2 $175 $298 $345 $114 $34 $55 $100 $77 $10 $38 $24 $676 $78 $94 $50 $90 $48 $55 $83 $102 $82 $119 $100 $106 $2 $52 $106 $9 $353 $28 $92 $26 $136 $110 $37 $121 $97 $69 $43 $17 $460 $41 $83 $288 $4 $86 $109 $101 $139 $113 $99 $103 $8 $122 $122 $111 $25 $107 $98 $10 $108 $63 $60 $164 $33 $82 $109 $41 $101 $94 $114 $104 $97 $9 $151 $426 $37 $11 $135 $5 $100 $25 $107 $277 $102 $182 $90 $169 $135 $11 $45 $38 $100 $119 $98 $21 $125 $72 $76 $32 $35 $60 $109 $61 $10 $2 $78 $12 $58 $95 $58 $106 $65 $65 $1 $8 $61 $110 $54 $107 $33 $91 $93 $364 $107 $92 $118 $44 $76 $19 $111 $122 $102 $61 $138 $76 $135 $116 $83 $113 $710 $120 $269 $101 $107 $125 $110 $124 $275 $101 $52 $4 $119 $26 $48 $53 $481 $141 $139 $64 $102 $116 $19 $77 $78 $103 $94 $99 $122 $1 $30 $65 $25 $133 $112 $113 

In [28]:
prices = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [33]:
np.random.seed(42)
vectorizer = CountVectorizer(max_features=2000, stop_words='english')
X = vectorizer.fit_transform(documents)

In [34]:
selected_words = vectorizer.get_feature_names_out()
print(f"Number of selected words: {len(selected_words)}")
print("Selected words: ", selected_words[1000:1030])

Number of selected words: 2000
Selected words:  ['jack' 'jacket' 'jeep' 'jet' 'jigsaw' 'joint' 'joints' 'kawasaki'
 'keeping' 'keeps' 'key' 'keyboard' 'keypad' 'keys' 'kg' 'khz' 'kia'
 'kickstand' 'kids' 'king' 'kingston' 'kit' 'kitchen' 'kitchens' 'knife'
 'knob' 'knobs' 'kohler' 'kraft' 'label']


In [35]:
regresor = LinearRegression()
regresor.fit(X, prices)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [36]:
def natural_language_linear_regression_pricer(item):
    x = vectorizer.transform([item.summary])
    return max(regresor.predict(x)[0], 0)

In [37]:
evaluate(natural_language_linear_regression_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$67 $124 $55 $6 $179 $211 $52 $50 $73 $7 $535 $186 $101 $150 $67 $96 $58 $49 $37 $29 $15 $87 $28 $9 $274 $241 $141 $13 $56 $80 $78 $57 $10 $43 $113 $418 $15 $65 $140 $71 $156 $61 $69 $26 $94 $59 $39 $49 $4 $13 $41 $91 $156 $31 $80 $69 $81 $166 $24 $12 $44 $18 $55 $24 $422 $93 $3 $312 $62 $212 $19 $33 $11 $130 $1 $36 $115 $36 $14 $130 $85 $59 $43 $42 $36 $117 $42 $105 $31 $177 $4 $106 $80 $27 $38 $91 $61 $1 $162 $102 $49 $56 $37 $15 $29 $4 $43 $214 $31 $98 $20 $20 $171 $42 $6 $149 $112 $25 $24 $40 $11 $108 $87 $28 $80 $27 $32 $127 $86 $29 $74 $40 $141 $23 $70 $55 $39 $103 $126 $76 $34 $171 $78 $95 $68 $91 $25 $235 $63 $32 $19 $189 $24 $27 $62 $49 $124 $28 $7 $9 $76 $26 $42 $13 $470 $22 $76 $50 $34 $9 $3 $23 $279 $27 $41 $144 $10 $40 $45 $144 $372 $18 $20 $13 $158 $32 $44 $16 $103 $36 $29 $122 $9 $27 $43 $60 $53 $6 $48 $18 

In [38]:
subset = 15000
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=4)
rf_model.fit(X[:subset], prices[:subset])

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [39]:
def random_forest(item):
    x = vectorizer.transform([item.summary])
    return max(0, rf_model.predict(x)[0])

In [40]:
evaluate(random_forest, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$75 $74 $4 $15 $112 $148 $78 $75 $33 $235 $521 $291 $58 $58 $16 $10 $50 $87 $30 $54 $33 $44 $18 $16 $148 $306 $169 $136 $39 $49 $44 $84 $68 $19 $18 $461 $47 $30 $144 $70 $137 $45 $8 $56 $141 $32 $57 $51 $54 $11 $15 $43 $244 $38 $295 $42 $60 $142 $4 $52 $101 $4 $59 $0 $453 $1 $22 $185 $15 $135 $12 $19 $23 $138 $0 $73 $108 $22 $31 $86 $62 $71 $55 $51 $13 $33 $12 $104 $8 $117 $27 $179 $2 $85 $47 $51 $10 $57 $136 $288 $35 $4 $19 $66 $41 $42 $110 $215 $18 $181 $40 $64 $74 $33 $88 $70 $81 $14 $226 $37 $34 $31 $41 $50 $31 $20 $47 $37 $96 $12 $25 $46 $80 $31 $44 $52 $36 $2 $84 $44 $4 $152 $5 $109 $114 $60 $25 $314 $84 $2 $20 $124 $47 $76 $52 $82 $109 $26 $41 $9 $17 $8 $3 $29 $384 $41 $273 $29 $17 $40 $21 $12 $220 $28 $8 $68 $55 $0 $40 $6 $333 $20 $75 $79 $143 $44 $61 $90 $8 $29 $35 $61 $24 $3 $11 $14 $57 $49 $4 $16 

In [43]:
np.random.seed(42)
xgb_model = xgb.XGBRegressor(n_estimators=1000, random_state=42, n_jobs=4, learning_rate=0.1)
xgb_model.fit(X, prices)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [44]:
def xg_boost(item):
    x = vectorizer.transform([item.summary])
    return max(0, xgb_model.predict(x)[0])

In [45]:
evaluate(xg_boost, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$95 $91 $24 $12 $167 $111 $18 $20 $22 $105 $359 $201 $138 $131 $16 $16 $44 $13 $31 $48 $4 $74 $14 $46 $171 $262 $146 $35 $50 $39 $78 $86 $23 $22 $33 $303 $19 $48 $163 $82 $130 $63 $54 $56 $126 $31 $73 $48 $35 $13 $23 $74 $133 $5 $61 $91 $54 $137 $17 $12 $28 $10 $46 $1 $408 $91 $6 $251 $58 $140 $18 $41 $79 $145 $8 $74 $173 $19 $34 $69 $74 $54 $39 $25 $21 $65 $52 $132 $13 $183 $23 $91 $36 $20 $28 $75 $26 $39 $153 $158 $32 $42 $3 $55 $3 $48 $51 $248 $31 $70 $39 $18 $62 $67 $43 $23 $43 $25 $53 $58 $59 $147 $86 $20 $86 $25 $5 $114 $136 $15 $27 $50 $90 $24 $72 $89 $84 $38 $43 $59 $15 $130 $40 $64 $49 $67 $37 $282 $77 $27 $19 $162 $22 $53 $30 $119 $128 $16 $14 $5 $99 $26 $5 $9 $391 $2 $112 $50 $16 $10 $29 $16 $197 $27 $30 $49 $27 $0 $31 $24 $253 $14 $189 $19 $157 $76 $56 $73 $21 $34 $89 $68 $1 $56 $10 $59 $12 $9 $36 $23 

In [48]:
import plotly.graph_objects as go

results = [
    ("Constant", "gray", 106.18),
    ("Linear Regression", "gray", 101.56),
    ("NLP + LR", "gray", 76.81),
    ("Random Forest", "gray", 73.04),
    ("XGBoost", "gray", 68.23),
]

labels, colors, values = zip(*results)

fig = go.Figure(go.Bar(x=labels, y=values, marker_color=colors))

fig.update_layout(
    title="Prediction error from each model",
    yaxis=dict(range=[0, max(values)], title="Error"),
    xaxis=dict(tickangle=45),
    width=1000,
    height=800
)

fig.show()